# 🌾 AgriCast360 Interactive PowerBI Dashboard Guide
## Complete Step-by-Step Implementation

**PowerBI Version:** 2.147.1085.0 64-bit (September 2025)

---

## 📊 Overview
This guide provides complete instructions to build an interactive agricultural price and weather analytics dashboard using the AgriCast360 dataset.

### Dataset Structure:
1. **Price_Data_Processed.csv** (14,967 rows)
   - Agricultural commodity prices across Gujarat markets
   - Columns: State, District, Market, Commodity, Variety, Grade, Arrival_Date, Min_Price, Max_Price, Modal_Price, Commodity_Code, Year, Month, Day, DayName, Quarter, Price_Range

2. **Weather CSV Files** (10 market-specific files in csv_exports/)
   - Bardoli, Kosamba, Mahuva, Mandvi, Nizar, Songadh, Surat, Uchhal, Valod, Vyra
   - Columns: market_name, latitude, longitude, time, temperature_2m_max, temperature_2m_min, temperature_2m_mean, precipitation_sum, rain_sum, wind_speed_10m_max, wind_gusts_10m_max, weather_code, cloud_cover_mean

3. **Market.xlsx** - Market reference/lookup table

---

## 📥 STEP 1: Data Import

### 1.1 Import Price Data
1. Open PowerBI Desktop
2. Click **Home** → **Get Data** → **Text/CSV**
3. Navigate to: `D:\CUDA_Experiments\Git_HUB\AgriCast360\Script\POWERBI FILES\Required Files\Price_Data_Processed.csv`
4. Click **Load**

### 1.2 Import Weather Data (All 10 Market Files)
1. Click **Home** → **Get Data** → **Folder**
2. Navigate to: `D:\CUDA_Experiments\Git_HUB\AgriCast360\Script\POWERBI FILES\Required Files\csv_exports`
3. Click **OK**
4. Click **Combine & Transform Data**
5. PowerBI will automatically combine all 10 CSV files
6. In Power Query Editor:
   - Remove the **Source.Name** column (if present)
   - Rename table to **Weather_Data**
7. Click **Close & Apply**

### 1.3 Import Market Reference Data
1. Click **Home** → **Get Data** → **Excel**
2. Navigate to: `D:\CUDA_Experiments\Git_HUB\AgriCast360\Script\POWERBI FILES\Required Files\Market.xlsx`
3. Select the appropriate sheet/table
4. Click **Load**

---

## 🔗 STEP 2: Data Modeling & Relationships

### 2.1 Create Date Table (Calendar Table)
1. Click **Home** → **Enter Data**
2. We'll create a calculated table instead. Go to **Modeling** → **New Table**
3. Enter this DAX:

```DAX
Calendar = 
ADDCOLUMNS(
    CALENDAR(DATE(2024,1,1), DATE(2024,12,31)),
    "Year", YEAR([Date]),
    "Month", FORMAT([Date], "MMM"),
    "MonthNumber", MONTH([Date]),
    "Quarter", "Q" & FORMAT([Date], "Q"),
    "DayName", FORMAT([Date], "dddd"),
    "DayOfWeek", WEEKDAY([Date]),
    "WeekNumber", WEEKNUM([Date]),
    "YearMonth", FORMAT([Date], "YYYY-MMM"),
    "IsWeekend", IF(WEEKDAY([Date]) IN {1,7}, "Weekend", "Weekday")
)
```

### 2.2 Create Relationships
1. Go to **Model View** (left sidebar icon)
2. Create relationships by dragging fields:

**Relationship 1: Price_Data ↔ Calendar**
- From: `Price_Data_Processed[Arrival_Date]`
- To: `Calendar[Date]`
- Cardinality: Many to One (*)
- Cross filter: Single

**Relationship 2: Weather_Data ↔ Calendar**
- Convert weather time field first in Power Query if needed
- From: `Weather_Data[time]`
- To: `Calendar[Date]`
- Cardinality: Many to One (*)
- Cross filter: Single

**Relationship 3: Price_Data ↔ Market.xlsx**
- From: `Price_Data_Processed[Market]`
- To: `Market[market_name]` (adjust field name as per your Market.xlsx)
- Cardinality: Many to One (*)

**Relationship 4: Weather_Data ↔ Market.xlsx**
- From: `Weather_Data[market_name]` (or market_name_lookup)
- To: `Market[market_name]`
- Cardinality: Many to One (*)

---

## 📊 STEP 3: Create Measures Table

### 3.1 Create Measures Table
1. Go to **Home** → **Enter Data**
2. Create empty table named **"_Measures"**
3. This will hold all our calculated measures

---

## 🧮 STEP 4: DAX MEASURES - Price Analytics

**Create each measure by:**
1. Click on **_Measures** table
2. Go to **Table Tools** → **New Measure**
3. Copy and paste each DAX formula below

### 4.1 Basic Price Measures

```DAX
Total_Records = COUNTROWS(Price_Data_Processed)
```

```DAX
Avg_Modal_Price = AVERAGE(Price_Data_Processed[Modal_Price])
```

```DAX
Avg_Min_Price = AVERAGE(Price_Data_Processed[Min_Price])
```

```DAX
Avg_Max_Price = AVERAGE(Price_Data_Processed[Max_Price])
```

```DAX
Total_Price_Range = SUM(Price_Data_Processed[Price_Range])
```

```DAX
Avg_Price_Range = AVERAGE(Price_Data_Processed[Price_Range])
```

### 4.2 Price Volatility Measures

```DAX
Price_Volatility = 
VAR AvgModalPrice = [Avg_Modal_Price]
VAR StdDev = STDEV.P(Price_Data_Processed[Modal_Price])
RETURN
IF(AvgModalPrice = 0, BLANK(), DIVIDE(StdDev, AvgModalPrice, 0) * 100)
```

```DAX
Coefficient_of_Variation = 
DIVIDE(
    STDEV.P(Price_Data_Processed[Modal_Price]),
    AVERAGE(Price_Data_Processed[Modal_Price]),
    0
) * 100
```

```DAX
Max_Price_Observed = MAX(Price_Data_Processed[Max_Price])
```

```DAX
Min_Price_Observed = MIN(Price_Data_Processed[Min_Price])
```

```DAX
Price_Range_Total_Span = [Max_Price_Observed] - [Min_Price_Observed]
```

### 4.3 Time Intelligence Measures

```DAX
Avg_Price_Previous_Month = 
CALCULATE(
    [Avg_Modal_Price],
    DATEADD(Calendar[Date], -1, MONTH)
)
```

```DAX
Price_MoM_Change = 
VAR CurrentPrice = [Avg_Modal_Price]
VAR PreviousPrice = [Avg_Price_Previous_Month]
RETURN
IF(
    ISBLANK(PreviousPrice) || PreviousPrice = 0,
    BLANK(),
    CurrentPrice - PreviousPrice
)
```

```DAX
Price_MoM_Change_Pct = 
VAR CurrentPrice = [Avg_Modal_Price]
VAR PreviousPrice = [Avg_Price_Previous_Month]
RETURN
IF(
    ISBLANK(PreviousPrice) || PreviousPrice = 0,
    BLANK(),
    DIVIDE(CurrentPrice - PreviousPrice, PreviousPrice, 0) * 100
)
```

```DAX
YTD_Avg_Price = 
CALCULATE(
    [Avg_Modal_Price],
    DATESYTD(Calendar[Date])
)
```

```DAX
QTD_Avg_Price = 
CALCULATE(
    [Avg_Modal_Price],
    DATESQTD(Calendar[Date])
)
```

### 4.4 Market Comparison Measures

```DAX
Market_Count = DISTINCTCOUNT(Price_Data_Processed[Market])
```

```DAX
Commodity_Count = DISTINCTCOUNT(Price_Data_Processed[Commodity])
```

```DAX
Highest_Avg_Price_Market = 
CALCULATE(
    FIRSTNONBLANK(Price_Data_Processed[Market], 1),
    TOPN(1, ALL(Price_Data_Processed[Market]), [Avg_Modal_Price], DESC)
)
```

```DAX
Lowest_Avg_Price_Market = 
CALCULATE(
    FIRSTNONBLANK(Price_Data_Processed[Market], 1),
    TOPN(1, ALL(Price_Data_Processed[Market]), [Avg_Modal_Price], ASC)
)
```

```DAX
Market_Price_Index = 
VAR OverallAvg = CALCULATE([Avg_Modal_Price], ALL(Price_Data_Processed[Market]))
VAR CurrentAvg = [Avg_Modal_Price]
RETURN
IF(OverallAvg = 0, BLANK(), DIVIDE(CurrentAvg, OverallAvg, 0) * 100)
```

### 4.5 Commodity Analysis Measures

```DAX
Price_per_Commodity = 
CALCULATE(
    [Avg_Modal_Price],
    ALLEXCEPT(Price_Data_Processed, Price_Data_Processed[Commodity])
)
```

```DAX
Commodity_Rank_by_Price = 
RANKX(
    ALL(Price_Data_Processed[Commodity]),
    [Avg_Modal_Price],
    ,
    DESC,
    DENSE
)
```

```DAX
Top_Commodity = 
CALCULATE(
    FIRSTNONBLANK(Price_Data_Processed[Commodity], 1),
    TOPN(1, ALL(Price_Data_Processed[Commodity]), [Avg_Modal_Price], DESC)
)
```

### 4.6 Quarter-Based Measures

```DAX
Avg_Price_by_Quarter = 
CALCULATE(
    [Avg_Modal_Price],
    ALLEXCEPT(Price_Data_Processed, Price_Data_Processed[Quarter])
)
```

```DAX
Q1_Avg_Price = 
CALCULATE(
    [Avg_Modal_Price],
    Price_Data_Processed[Quarter] = 1
)
```

```DAX
Q2_Avg_Price = 
CALCULATE(
    [Avg_Modal_Price],
    Price_Data_Processed[Quarter] = 2
)
```

```DAX
Q3_Avg_Price = 
CALCULATE(
    [Avg_Modal_Price],
    Price_Data_Processed[Quarter] = 3
)
```

```DAX
Q4_Avg_Price = 
CALCULATE(
    [Avg_Modal_Price],
    Price_Data_Processed[Quarter] = 4
)
```

### 4.7 Advanced Price Analytics

```DAX
Median_Modal_Price = 
MEDIAN(Price_Data_Processed[Modal_Price])
```

```DAX
Price_Percentile_25 = 
PERCENTILE.INC(Price_Data_Processed[Modal_Price], 0.25)
```

```DAX
Price_Percentile_75 = 
PERCENTILE.INC(Price_Data_Processed[Modal_Price], 0.75)
```

```DAX
IQR_Price_Range = 
[Price_Percentile_75] - [Price_Percentile_25]
```

```DAX
Arrival_Days_Count = DISTINCTCOUNT(Price_Data_Processed[Arrival_Date])
```

```DAX
Price_Growth_Rate = 
VAR FirstPrice = CALCULATE([Avg_Modal_Price], FIRSTDATE(Calendar[Date]))
VAR LastPrice = CALCULATE([Avg_Modal_Price], LASTDATE(Calendar[Date]))
RETURN
IF(
    FirstPrice = 0 || ISBLANK(FirstPrice),
    BLANK(),
    DIVIDE(LastPrice - FirstPrice, FirstPrice, 0) * 100
)
```

---

## 🌤️ STEP 5: DAX MEASURES - Weather Analytics

### 5.1 Temperature Measures

```DAX
Avg_Temperature_Max = AVERAGE(Weather_Data[temperature_2m_max])
```

```DAX
Avg_Temperature_Min = AVERAGE(Weather_Data[temperature_2m_min])
```

```DAX
Avg_Temperature_Mean = AVERAGE(Weather_Data[temperature_2m_mean])
```

```DAX
Temperature_Range = 
[Avg_Temperature_Max] - [Avg_Temperature_Min]
```

```DAX
Max_Temperature_Recorded = MAX(Weather_Data[temperature_2m_max])
```

```DAX
Min_Temperature_Recorded = MIN(Weather_Data[temperature_2m_min])
```

```DAX
Temperature_Fluctuation = 
STDEV.P(Weather_Data[temperature_2m_mean])
```

### 5.2 Precipitation Measures

```DAX
Total_Precipitation = SUM(Weather_Data[precipitation_sum])
```

```DAX
Total_Rainfall = SUM(Weather_Data[rain_sum])
```

```DAX
Avg_Daily_Precipitation = AVERAGE(Weather_Data[precipitation_sum])
```

```DAX
Rainy_Days_Count = 
CALCULATE(
    COUNTROWS(Weather_Data),
    Weather_Data[rain_sum] > 0
)
```

```DAX
Rainy_Days_Percentage = 
VAR TotalDays = COUNTROWS(Weather_Data)
VAR RainyDays = [Rainy_Days_Count]
RETURN
IF(TotalDays = 0, BLANK(), DIVIDE(RainyDays, TotalDays, 0) * 100)
```

```DAX
Max_Precipitation_Day = MAX(Weather_Data[precipitation_sum])
```

```DAX
Dry_Days_Count = 
CALCULATE(
    COUNTROWS(Weather_Data),
    Weather_Data[rain_sum] = 0
)
```

### 5.3 Wind Measures

```DAX
Avg_Wind_Speed = AVERAGE(Weather_Data[wind_speed_10m_max])
```

```DAX
Avg_Wind_Gusts = AVERAGE(Weather_Data[wind_gusts_10m_max])
```

```DAX
Max_Wind_Speed = MAX(Weather_Data[wind_speed_10m_max])
```

```DAX
Max_Wind_Gust = MAX(Weather_Data[wind_gusts_10m_max])
```

```DAX
High_Wind_Days = 
CALCULATE(
    COUNTROWS(Weather_Data),
    Weather_Data[wind_speed_10m_max] > 20
)
```

```DAX
Wind_Intensity_Index = 
VAR AvgWind = [Avg_Wind_Speed]
VAR AvgGust = [Avg_Wind_Gusts]
RETURN
(AvgWind + AvgGust) / 2
```

### 5.4 Cloud Cover Measures

```DAX
Avg_Cloud_Cover = AVERAGE(Weather_Data[cloud_cover_mean])
```

```DAX
Clear_Days = 
CALCULATE(
    COUNTROWS(Weather_Data),
    Weather_Data[cloud_cover_mean] < 25
)
```

```DAX
Cloudy_Days = 
CALCULATE(
    COUNTROWS(Weather_Data),
    Weather_Data[cloud_cover_mean] > 75
)
```

```DAX
Partly_Cloudy_Days = 
CALCULATE(
    COUNTROWS(Weather_Data),
    Weather_Data[cloud_cover_mean] >= 25 && Weather_Data[cloud_cover_mean] <= 75
)
```

### 5.5 Weather Code Analysis

```DAX
Most_Common_Weather = 
CALCULATE(
    FIRSTNONBLANK(Weather_Data[weather_code], 1),
    TOPN(1, ALL(Weather_Data[weather_code]), COUNTROWS(Weather_Data), DESC)
)
```

```DAX
Weather_Variation_Count = 
DISTINCTCOUNT(Weather_Data[weather_code])
```

### 5.6 Market-Specific Weather Measures

```DAX
Weather_Market_Count = 
DISTINCTCOUNT(Weather_Data[market_name])
```

```DAX
Hottest_Market = 
CALCULATE(
    FIRSTNONBLANK(Weather_Data[market_name], 1),
    TOPN(1, ALL(Weather_Data[market_name]), [Avg_Temperature_Max], DESC)
)
```

```DAX
Coldest_Market = 
CALCULATE(
    FIRSTNONBLANK(Weather_Data[market_name], 1),
    TOPN(1, ALL(Weather_Data[market_name]), [Avg_Temperature_Min], ASC)
)
```

```DAX
Wettest_Market = 
CALCULATE(
    FIRSTNONBLANK(Weather_Data[market_name], 1),
    TOPN(1, ALL(Weather_Data[market_name]), [Total_Rainfall], DESC)
)
```

### 5.7 Weather Severity Indicators

```DAX
Extreme_Heat_Days = 
CALCULATE(
    COUNTROWS(Weather_Data),
    Weather_Data[temperature_2m_max] > 35
)
```

```DAX
Cold_Days = 
CALCULATE(
    COUNTROWS(Weather_Data),
    Weather_Data[temperature_2m_min] < 15
)
```

```DAX
Heavy_Rain_Days = 
CALCULATE(
    COUNTROWS(Weather_Data),
    Weather_Data[rain_sum] > 10
)
```

```DAX
Weather_Comfort_Index = 
VAR TempComfort = IF([Avg_Temperature_Mean] >= 20 && [Avg_Temperature_Mean] <= 30, 1, 0)
VAR RainComfort = IF([Avg_Daily_Precipitation] < 2, 1, 0)
VAR WindComfort = IF([Avg_Wind_Speed] < 15, 1, 0)
RETURN
(TempComfort + RainComfort + WindComfort) / 3 * 100
```

---

## 📈 STEP 6: DAX MEASURES - Price-Weather Correlation

### 6.1 Correlation Measures

```DAX
Price_Temp_Correlation_Score = 
VAR PriceTable = 
    SUMMARIZE(
        FILTER(
            ALL(Calendar),
            NOT(ISBLANK([Avg_Modal_Price])) && NOT(ISBLANK([Avg_Temperature_Mean]))
        ),
        Calendar[Date],
        "Price", [Avg_Modal_Price],
        "Temp", [Avg_Temperature_Mean]
    )
VAR N = COUNTROWS(PriceTable)
VAR SumPrice = SUMX(PriceTable, [Price])
VAR SumTemp = SUMX(PriceTable, [Temp])
VAR SumPriceTemp = SUMX(PriceTable, [Price] * [Temp])
VAR SumPriceSquare = SUMX(PriceTable, [Price] * [Price])
VAR SumTempSquare = SUMX(PriceTable, [Temp] * [Temp])
VAR Numerator = (N * SumPriceTemp) - (SumPrice * SumTemp)
VAR Denominator = SQRT((N * SumPriceSquare - SumPrice * SumPrice) * (N * SumTempSquare - SumTemp * SumTemp))
RETURN
IF(Denominator = 0, BLANK(), DIVIDE(Numerator, Denominator, 0))
```

```DAX
Weather_Impact_on_Price = 
VAR HighTempDays = CALCULATE([Avg_Modal_Price], [Avg_Temperature_Max] > 30)
VAR NormalDays = CALCULATE([Avg_Modal_Price], [Avg_Temperature_Max] <= 30)
RETURN
IF(NormalDays = 0, BLANK(), DIVIDE(HighTempDays - NormalDays, NormalDays, 0) * 100)
```

```DAX
Rain_Impact_on_Price = 
VAR RainyDayPrice = CALCULATE([Avg_Modal_Price], Weather_Data[rain_sum] > 0)
VAR DryDayPrice = CALCULATE([Avg_Modal_Price], Weather_Data[rain_sum] = 0)
RETURN
IF(DryDayPrice = 0, BLANK(), DIVIDE(RainyDayPrice - DryDayPrice, DryDayPrice, 0) * 100)
```

### 6.2 Conditional Format Measures (for visuals)

```DAX
Price_Trend_Indicator = 
VAR Change = [Price_MoM_Change_Pct]
RETURN
SWITCH(
    TRUE(),
    Change > 5, "Strong Increase ↑",
    Change > 0, "Increase ↑",
    Change = 0, "Stable →",
    Change > -5, "Decrease ↓",
    "Strong Decrease ↓"
)
```

```DAX
Weather_Severity_Rating = 
VAR TempScore = IF([Avg_Temperature_Max] > 35, 1, 0)
VAR RainScore = IF([Total_Rainfall] > 100, 1, 0)
VAR WindScore = IF([Avg_Wind_Speed] > 20, 1, 0)
VAR Total = TempScore + RainScore + WindScore
RETURN
SWITCH(
    Total,
    3, "Severe",
    2, "High",
    1, "Moderate",
    "Normal"
)
```

```DAX
Price_Status = 
VAR CurrentPrice = [Avg_Modal_Price]
VAR OverallMedian = [Median_Modal_Price]
RETURN
IF(
    CurrentPrice > OverallMedian * 1.2, "High",
    IF(CurrentPrice < OverallMedian * 0.8, "Low", "Normal")
)
```

### 6.3 KPI Measures

```DAX
Price_KPI = 
VAR Target = 3000
VAR Actual = [Avg_Modal_Price]
RETURN
DIVIDE(Actual, Target, 0) - 1
```

```DAX
Market_Performance_Score = 
VAR PriceScore = IF([Avg_Modal_Price] > [Median_Modal_Price], 60, 40)
VAR VolatilityScore = IF([Price_Volatility] < 20, 20, 10)
VAR GrowthScore = IF([Price_Growth_Rate] > 0, 20, 0)
RETURN
PriceScore + VolatilityScore + GrowthScore
```

---

## 📊 STEP 7: Build Dashboard Visualizations

### Dashboard Layout Strategy
Create a multi-page dashboard with these pages:
1. **Overview Dashboard** - Key metrics and trends
2. **Price Analytics** - Detailed price analysis
3. **Weather Analytics** - Weather patterns
4. **Market Comparison** - Cross-market analysis
5. **Commodity Deep Dive** - Individual commodity focus

---

## 📄 PAGE 1: OVERVIEW DASHBOARD

### Visual 1: KPI Cards (Top Row)
**Type:** Card Visual (5 cards in a row)

1. **Card 1: Average Modal Price**
   - Field: `[Avg_Modal_Price]`
   - Format: Currency, ₹ symbol
   - Font: Bold, 32pt

2. **Card 2: Total Markets**
   - Field: `[Market_Count]`
   - Format: Whole number

3. **Card 3: Total Commodities**
   - Field: `[Commodity_Count]`
   - Format: Whole number

4. **Card 4: Price MoM Change %**
   - Field: `[Price_MoM_Change_Pct]`
   - Format: Percentage, 2 decimals
   - Conditional formatting: Green if positive, Red if negative

5. **Card 5: Average Temperature**
   - Field: `[Avg_Temperature_Mean]`
   - Format: Decimal, 1 decimal, add " °C" suffix

### Visual 2: Price Trend Over Time
**Type:** Line Chart
- **X-Axis:** `Calendar[Date]` (Date hierarchy: Year > Quarter > Month > Day)
- **Y-Axis:** 
  - `[Avg_Modal_Price]` (Primary)
  - `[Avg_Min_Price]` (Primary)
  - `[Avg_Max_Price]` (Primary)
- **Legend:** Show
- **Data Labels:** Off
- **Markers:** On
- **Line Style:** Smooth
- **Colors:** Blue (Modal), Green (Min), Red (Max)

### Visual 3: Market Performance Matrix
**Type:** Table
- **Columns:**
  - `Price_Data_Processed[Market]`
  - `[Avg_Modal_Price]`
  - `[Price_Volatility]`
  - `[Market_Performance_Score]`
  - `[Price_MoM_Change_Pct]`
- **Sort By:** `[Market_Performance_Score]` descending
- **Conditional Formatting:**
  - Price_Volatility: Red gradient (high volatility)
  - Market_Performance_Score: Green gradient
  - Price_MoM_Change_Pct: Red/Green based on value

### Visual 4: Commodity Price Distribution
**Type:** Clustered Column Chart
- **X-Axis:** `Price_Data_Processed[Commodity]`
- **Y-Axis:** `[Avg_Modal_Price]`
- **Data Labels:** On
- **Sort:** Descending by value
- **Colors:** Gradient (low to high)

### Visual 5: Weather Overview Gauge
**Type:** Gauge Chart (3 gauges)

1. **Temperature Gauge**
   - Value: `[Avg_Temperature_Mean]`
   - Min: 0, Max: 50
   - Target: 25

2. **Rainfall Gauge**
   - Value: `[Total_Rainfall]`
   - Min: 0, Max: 500

3. **Wind Speed Gauge**
   - Value: `[Avg_Wind_Speed]`
   - Min: 0, Max: 40

### Visual 6: Top 5 Markets by Price
**Type:** Bar Chart (Horizontal)
- **Y-Axis:** `Price_Data_Processed[Market]`
- **X-Axis:** `[Avg_Modal_Price]`
- **Top N Filter:** Top 5 by `[Avg_Modal_Price]`
- **Data Labels:** On (inside)
- **Color:** Single color (green)

---

## 📄 PAGE 2: PRICE ANALYTICS

### Visual 1: Price Volatility Analysis
**Type:** Line and Clustered Column Chart (Combo)
- **Shared X-Axis:** `Calendar[YearMonth]`
- **Column Y-Axis:** `[Avg_Modal_Price]`
- **Line Y-Axis:** `[Price_Volatility]` (Secondary axis)
- **Data Labels:** On for both
- **Line Color:** Orange
- **Column Color:** Blue

### Visual 2: Price Distribution Box Plot
**Type:** Box and Whisker Chart (use custom visual if needed)
- **Category:** `Price_Data_Processed[Market]`
- **Values:** `Price_Data_Processed[Modal_Price]`
- **Alternative:** Use scatter plot with:
  - X: Market
  - Y: `[Price_Percentile_25]`, `[Median_Modal_Price]`, `[Price_Percentile_75]`

### Visual 3: Quarter-wise Price Comparison
**Type:** Clustered Bar Chart
- **Y-Axis:** `Price_Data_Processed[Quarter]`
- **X-Axis:** 
  - `[Q1_Avg_Price]`
  - `[Q2_Avg_Price]`
  - `[Q3_Avg_Price]`
  - `[Q4_Avg_Price]`
- **Legend:** Show quarters
- **Data Labels:** On

### Visual 4: Price Range Analysis
**Type:** Area Chart
- **X-Axis:** `Calendar[Date]`
- **Y-Axis:**
  - `[Max_Price_Observed]` (top line)
  - `[Avg_Modal_Price]` (middle)
  - `[Min_Price_Observed]` (bottom)
- **Area Fill:** Gradient between lines
- **Transparency:** 50%

### Visual 5: Commodity Rank Table
**Type:** Matrix
- **Rows:** `Price_Data_Processed[Commodity]`
- **Values:**
  - `[Avg_Modal_Price]`
  - `[Commodity_Rank_by_Price]`
  - `[Price_Volatility]`
  - `[Total_Records]`
- **Conditional Formatting:**
  - Rank: Color scale (1=green, last=red)
  - Price: Data bars
  - Volatility: Icons (high/medium/low)

### Visual 6: Daily Price Trend with Moving Average
**Type:** Line Chart
- **X-Axis:** `Calendar[Date]`
- **Y-Axis:** `[Avg_Modal_Price]`
- **Trend Line:** Add moving average (30 days)
- **To add trend line:**
  1. Click on visual
  2. Analytics pane
  3. Add Trend Line > Moving Average
  4. Set points: 30

### Visual 7: Price Growth Rate by Market
**Type:** Funnel Chart
- **Category:** `Price_Data_Processed[Market]`
- **Values:** `[Price_Growth_Rate]`
- **Data Labels:** On
- **Sort:** Descending

### Visual 8: Price Status Donut Chart
**Type:** Donut Chart
- **Legend:** `[Price_Status]` (High/Normal/Low)
- **Values:** `[Total_Records]`
- **Detail Labels:** Percentage and value
- **Colors:** Red (High), Green (Normal), Blue (Low)

---

## 📄 PAGE 3: WEATHER ANALYTICS

### Visual 1: Temperature Trend Analysis
**Type:** Line Chart with Multiple Lines
- **X-Axis:** `Calendar[Date]`
- **Y-Axis:**
  - `[Avg_Temperature_Max]` (Red line)
  - `[Avg_Temperature_Mean]` (Orange line)
  - `[Avg_Temperature_Min]` (Blue line)
- **Markers:** On
- **Line Width:** 2
- **Legend:** Show

### Visual 2: Precipitation Calendar Heatmap
**Type:** Matrix with Conditional Formatting
- **Rows:** `Calendar[Month]`
- **Columns:** `Calendar[Day]`
- **Values:** `[Avg_Daily_Precipitation]`
- **Conditional Formatting:**
  - Background color scale: White (0) to Blue (max)
  - Font: White for dark cells

### Visual 3: Weather Conditions Distribution
**Type:** Clustered Column Chart
- **X-Axis:** Weather condition categories
  - `[Clear_Days]`
  - `[Partly_Cloudy_Days]`
  - `[Cloudy_Days]`
  - `[Rainy_Days_Count]`
- **Y-Axis:** Count of days
- **Data Labels:** On
- **Colors:** Yellow, Gray, Dark Gray, Blue

### Visual 4: Wind Speed Analysis
**Type:** Scatter Chart
- **X-Axis:** `[Avg_Wind_Speed]`
- **Y-Axis:** `[Avg_Wind_Gusts]`
- **Details:** `Weather_Data[market_name]`
- **Size:** `[Wind_Intensity_Index]`
- **Play Axis:** `Calendar[Date]` (for animation)

### Visual 5: Market Temperature Comparison
**Type:** Stacked Bar Chart
- **Y-Axis:** `Weather_Data[market_name]`
- **X-Axis:**
  - `[Extreme_Heat_Days]` (Red)
  - `[Cold_Days]` (Blue)
  - Normal days (calculated: Total - Extreme - Cold) (Green)
- **Data Labels:** On
- **Legend:** Show

### Visual 6: Rainfall Distribution by Market
**Type:** Waterfall Chart
- **Category:** `Weather_Data[market_name]`
- **Y-Axis:** `[Total_Rainfall]`
- **Sort:** Descending
- **Colors:** Increase (Blue), Decrease (Gray)

### Visual 7: Weather Severity Rating
**Type:** Gauge Chart
- **Value:** `[Weather_Comfort_Index]`
- **Min:** 0
- **Max:** 100
- **Target:** 75
- **Color Bands:**
  - 0-40: Red (Poor)
  - 40-70: Yellow (Moderate)
  - 70-100: Green (Good)

### Visual 8: Cloud Cover vs Temperature
**Type:** Scatter Chart
- **X-Axis:** `[Avg_Cloud_Cover]`
- **Y-Axis:** `[Avg_Temperature_Mean]`
- **Details:** `Weather_Data[market_name]`
- **Size:** `[Total_Precipitation]`
- **Color Saturation:** `Calendar[Month]`

### Visual 9: Monthly Weather Summary Table
**Type:** Table
- **Columns:**
  - `Calendar[Month]`
  - `[Avg_Temperature_Mean]`
  - `[Total_Rainfall]`
  - `[Rainy_Days_Count]`
  - `[Avg_Wind_Speed]`
  - `[Weather_Comfort_Index]`
- **Sort:** By Month
- **Conditional Formatting:**
  - Temperature: Red gradient
  - Rainfall: Blue gradient
  - Comfort Index: Green gradient

---

## 📄 PAGE 4: MARKET COMPARISON

### Visual 1: Market Performance Dashboard
**Type:** Multi-Row Card
- **Fields:**
  - `Price_Data_Processed[Market]`
  - `[Avg_Modal_Price]`
  - `[Market_Performance_Score]`
  - `[Price_Volatility]`
- **Grouping:** By Market
- **Sort:** By Performance Score descending

### Visual 2: Price Comparison Across Markets
**Type:** Line Chart
- **X-Axis:** `Calendar[Date]`
- **Legend:** `Price_Data_Processed[Market]`
- **Y-Axis:** `[Avg_Modal_Price]`
- **Play Axis:** `Calendar[MonthNumber]`
- **Line Style:** Smooth
- **Data Labels:** Off (too cluttered)
- **Tooltip:** Show all market values

### Visual 3: Market Price Index
**Type:** Decomposition Tree
- **Root:** All Markets
- **Analyze:** `[Market_Price_Index]`
- **Explain By:**
  - `Price_Data_Processed[Market]`
  - `Price_Data_Processed[Commodity]`
  - `Calendar[Quarter]`

### Visual 4: Geographic Market Distribution
**Type:** Map Visual
- **Location:** Use Market table coordinates (latitude/longitude)
- **Size:** `[Avg_Modal_Price]`
- **Color Saturation:** `[Market_Performance_Score]`
- **Tooltip:**
  - Market Name
  - `[Avg_Modal_Price]`
  - `[Total_Records]`
  - `[Price_Volatility]`

### Visual 5: Market Correlation Matrix
**Type:** Matrix with Conditional Formatting
- **Rows:** `Price_Data_Processed[Market]`
- **Columns:** Measure names:
  - `[Avg_Modal_Price]`
  - `[Price_Volatility]`
  - `[Avg_Temperature_Mean]`
  - `[Total_Rainfall]`
  - `[Market_Performance_Score]`
- **Conditional Formatting:** Heat map (Red-Yellow-Green)

### Visual 6: Top vs Bottom Markets
**Type:** Clustered Bar Chart
- **Y-Axis:** Market Name
- **X-Axis:** `[Avg_Modal_Price]`
- **Filters:**
  - Top 5 Markets (one chart)
  - Bottom 5 Markets (separate chart below)
- **Colors:** Green (Top), Red (Bottom)

### Visual 7: Market Volatility Bubble Chart
**Type:** Scatter Chart
- **X-Axis:** `[Avg_Modal_Price]`
- **Y-Axis:** `[Price_Volatility]`
- **Details:** `Price_Data_Processed[Market]`
- **Size:** `[Total_Records]`
- **Color:** `[Market_Performance_Score]` (gradient)
- **Quadrant Lines:**
  - Add reference lines at median values

### Visual 8: Market Ranking Table
**Type:** Table with Sparklines
- **Columns:**
  - Rank (calculated)
  - `Price_Data_Processed[Market]`
  - `[Avg_Modal_Price]`
  - Price Trend (mini line chart)
  - `[Price_MoM_Change_Pct]`
  - `[Market_Performance_Score]`
- **Sort:** By Performance Score
- **Conditional Icons:**
  - Up/Down arrows for price change

---

## 📄 PAGE 5: COMMODITY DEEP DIVE

### Visual 1: Commodity Selection Slicer
**Type:** Slicer (Dropdown or Tiles)
- **Field:** `Price_Data_Processed[Commodity]`
- **Style:** Dropdown list
- **Multi-select:** Off (single selection)
- **Position:** Top of page

### Visual 2: Selected Commodity KPIs
**Type:** Card Visuals (4 cards)
1. Current Average Price: `[Avg_Modal_Price]`
2. Price Volatility: `[Price_Volatility]`
3. Market Count: `[Market_Count]`
4. Growth Rate: `[Price_Growth_Rate]`
- **Note:** These will dynamically change based on slicer selection

### Visual 3: Commodity Price Timeline
**Type:** Area Chart
- **X-Axis:** `Calendar[Date]`
- **Y-Axis:** `[Avg_Modal_Price]`
- **Legend:** `Price_Data_Processed[Variety]` (if multiple varieties)
- **Shading:** Gradient fill
- **Tooltip:** Enhanced with all measures

### Visual 4: Price Distribution by Market
**Type:** Box Plot or Violin Chart
- **Category:** `Price_Data_Processed[Market]`
- **Values:** `Price_Data_Processed[Modal_Price]`
- **Shows:** Distribution spread per market
- **Alternative:** Use Column chart with error bars

### Visual 5: Min-Max-Modal Price Comparison
**Type:** Clustered Column Chart
- **X-Axis:** `Calendar[Month]`
- **Y-Axis:**
  - `[Avg_Min_Price]` (Green)
  - `[Avg_Modal_Price]` (Blue)
  - `[Avg_Max_Price]` (Red)
- **Legend:** Show
- **Data Labels:** On

### Visual 6: Commodity Weather Correlation
**Type:** Line and Stacked Column Combo
- **Shared X-Axis:** `Calendar[Date]`
- **Column Y-Axis:** `[Avg_Modal_Price]` (Primary)
- **Line Y-Axis:** `[Avg_Temperature_Mean]` (Secondary)
- **Second Line:** `[Total_Rainfall]` (Secondary, different scale)
- **Colors:** Price (Blue), Temp (Orange), Rain (Light Blue)

### Visual 7: Day of Week Analysis
**Type:** Column Chart
- **X-Axis:** `Price_Data_Processed[DayName]`
- **Y-Axis:** `[Avg_Modal_Price]`
- **Sort:** Custom order (Mon-Sun)
- **Data Labels:** On
- **Insight:** Shows if certain days have higher prices

### Visual 8: Quarterly Performance
**Type:** Waterfall Chart
- **Category:** Quarter progression (Q1→Q2→Q3→Q4)
- **Y-Axis:** `[Avg_Modal_Price]`
- **Shows:** How price changes quarter over quarter
- **Color:** Green (increase), Red (decrease)

### Visual 9: Commodity Market Matrix
**Type:** Matrix
- **Rows:** `Price_Data_Processed[Market]`
- **Columns:** `Calendar[Month]`
- **Values:** `[Avg_Modal_Price]`
- **Conditional Formatting:** Color scale (low=green, high=red)
- **Heat map style**

### Visual 10: Price Trend Indicator
**Type:** Card with Conditional Formatting
- **Value:** `[Price_Trend_Indicator]`
- **Font Size:** Large
- **Conditional Formatting:**
  - Strong Increase: Green background
  - Increase: Light green
  - Stable: Yellow
  - Decrease: Light red
  - Strong Decrease: Red background

---

## 🎛️ STEP 8: Add Interactivity & Filters

### 8.1 Global Filters (Apply to All Pages)

**Create Slicers on Each Page:**

1. **Date Range Slicer**
   - Type: Date Slicer (Between)
   - Field: `Calendar[Date]`
   - Style: Slider
   - Position: Top-left of each page
   - Sync across pages: Yes

2. **Market Slicer**
   - Type: Slicer (Dropdown or List)
   - Field: `Price_Data_Processed[Market]`
   - Multi-select: Yes
   - Search box: Yes (for 10 markets)
   - Position: Top-center

3. **Commodity Slicer**
   - Type: Slicer (Dropdown)
   - Field: `Price_Data_Processed[Commodity]`
   - Multi-select: Yes
   - Position: Top-right

### 8.2 Sync Slicers Across Pages

1. Select a slicer
2. Click **View** → **Sync Slicers**
3. In Sync Slicers pane:
   - Check pages where slicer should appear
   - Check sync icon for pages where filtering should apply
4. Repeat for Date, Market, and Commodity slicers

### 8.3 Drill-Through Pages

**Create Drill-Through from Overview to Commodity Deep Dive:**

1. Go to **PAGE 5: Commodity Deep Dive**
2. In Fields pane, drag `Price_Data_Processed[Commodity]` to **Drill through** area
3. Check "Keep all filters"
4. Now users can right-click any commodity → **Drill through** → Commodity Deep Dive

**Create Drill-Through for Market Details:**

1. Create a new page: **"Market Detail"**
2. Add detailed visuals for single market
3. Add drill-through filter: `Price_Data_Processed[Market]`

### 8.4 Tooltips (Custom Report Tooltips)

**Create Custom Tooltip Page:**

1. Create new page: **"Price Tooltip"**
2. **Page Settings:**
   - Page Size: Tooltip
   - Allow use as tooltip: Yes
3. Add small visuals:
   - Card: `[Avg_Modal_Price]`
   - Mini line chart: Price trend (last 30 days)
   - Card: `[Price_MoM_Change_Pct]`

4. **Apply Tooltip to Visuals:**
   - Select any price chart
   - Format pane → Tooltip → Report page: "Price Tooltip"

### 8.5 Bookmarks for Views

**Create Bookmarks for Different Scenarios:**

1. **View 1: High Price Alert**
   - Filter: Price > Median * 1.2
   - Create bookmark: View → Bookmarks → Add
   - Name: "High Price Markets"

2. **View 2: Volatile Markets**
   - Filter: Price_Volatility > 20
   - Create bookmark: "High Volatility"

3. **View 3: Adverse Weather**
   - Filter: Extreme_Heat_Days > 10 OR Heavy_Rain_Days > 5
   - Create bookmark: "Adverse Weather Conditions"

**Add Bookmark Navigator:**
- Insert → Buttons → Bookmark Navigator
- Position on Overview page

### 8.6 Cross-Filtering Settings

**Configure Cross-Filter Behavior:**

1. Select a visual
2. Format pane → Edit interactions (Home tab)
3. Set cross-filter behavior:
   - **Price charts ↔ Market slicers:** Filter (default)
   - **Weather charts ↔ Price charts:** Highlight only
   - **KPI cards:** No interaction (keep showing totals)

**Disable interactions for KPI cards:**
- Select any chart
- Click **Format** → **Edit interactions**
- Click "None" icon above KPI cards

### 8.7 Dynamic Titles Using Measures

**Create Dynamic Title Measure:**

```DAX
Dynamic_Title = 
VAR SelectedMarket = SELECTEDVALUE(Price_Data_Processed[Market], "All Markets")
VAR SelectedCommodity = SELECTEDVALUE(Price_Data_Processed[Commodity], "All Commodities")
VAR CurrentPrice = [Avg_Modal_Price]
RETURN
"Price Analysis: " & SelectedCommodity & " in " & SelectedMarket & 
" | Avg Price: ₹" & FORMAT(CurrentPrice, "#,##0")
```

**Apply to Chart Title:**
1. Select visual
2. Format visual → Title → Text: fx (conditional formatting)
3. Format by: Field value
4. Select: `[Dynamic_Title]`

### 8.8 Hierarchies for Drill-Down

**Create Date Hierarchy:**
1. Right-click `Calendar[Year]`
2. Create hierarchy → Name: "Date Hierarchy"
3. Add levels: Year → Quarter → Month → Date
4. Use in visuals for drill-down capability

**Create Location Hierarchy:**
1. Right-click `Price_Data_Processed[State]`
2. Create hierarchy → Name: "Location"
3. Add levels: State → District → Market

---

## 🎨 STEP 9: Formatting & Design

### 9.1 Apply Theme

**Built-in Themes:**
1. Go to **View** → **Themes**
2. Select: **Solar** or **Executive** (recommended for agriculture dashboard)

**Custom Colors for Agriculture:**
- Primary: #2E7D32 (Green)
- Secondary: #FF6F00 (Orange)
- Accent: #0288D1 (Blue)
- Alert: #D32F2F (Red)

### 9.2 Format Visual Elements

**Cards:**
- Font: Segoe UI Bold, 28-32pt
- Background: Light gray (#F5F5F5)
- Shadow: Subtle

**Charts:**
- Title: 14pt Bold
- Data labels: 10pt
- Grid lines: Light gray, thin
- Background: White

**Tables/Matrix:**
- Header: Bold, colored background
- Alternating rows: Yes
- Grid lines: Horizontal only

### 9.3 Add Report Header

1. Insert → Text Box
2. Text: **"AgriCast360 - Agricultural Price & Weather Analytics"**
3. Format: Segoe UI Bold 18pt, Dark green (#1B5E20)
4. Position: Top of page, full width

### 9.4 Page Navigation Buttons

1. Insert → Buttons → Blank
2. Action: Page navigation
3. Format: Green gradient, White text, Rounded corners
4. Create buttons for all pages

### 9.5 Conditional Formatting

**Price-Based:**
- Low (< 25th percentile): Green
- Medium (25-75th): Yellow
- High (> 75th): Red

**Apply:** Select visual → Format → Conditional formatting → Set rules

---

## ✅ STEP 10: Testing Checklist

### Data Validation
- [ ] All files loaded (10 weather + 1 price + 1 market)
- [ ] Relationships active in Model View
- [ ] Date formats consistent

### Measures Validation
- [ ] All 70+ measures created in _Measures table
- [ ] No #ERROR or BLANK values
- [ ] Time intelligence working (MoM, YTD, QTD)

### Visual Testing
- [ ] All KPI cards displaying
- [ ] Charts rendering smoothly
- [ ] Tables sorting correctly
- [ ] Conditional formatting visible
- [ ] Data labels readable

### Interactivity Testing
- [ ] Slicers filtering correctly
- [ ] Cross-filtering working
- [ ] Drill-through functioning
- [ ] Bookmarks working
- [ ] Navigation buttons working

### Performance
- [ ] Dashboard loads in < 5 seconds
- [ ] Quick slicer response
- [ ] No lag when switching pages

---

## 📊 STEP 11: Measures Summary

### Total: 70+ DAX Measures Created

**Price Analysis (29 measures)**
- Basic: Avg_Modal_Price, Avg_Min_Price, Avg_Max_Price, Total_Price_Range, Median_Modal_Price
- Volatility: Price_Volatility, Coefficient_of_Variation, IQR_Price_Range
- Time Intelligence: Price_MoM_Change, Price_MoM_Change_Pct, YTD_Avg_Price, QTD_Avg_Price
- Market: Market_Count, Commodity_Count, Market_Price_Index, Commodity_Rank_by_Price
- Quarters: Q1_Avg_Price, Q2_Avg_Price, Q3_Avg_Price, Q4_Avg_Price
- Analytics: Price_Growth_Rate, Market_Performance_Score, Price_Status, Price_Trend_Indicator

**Weather Analysis (28 measures)**
- Temperature: Avg_Temperature_Max, Avg_Temperature_Min, Avg_Temperature_Mean, Temperature_Range
- Precipitation: Total_Rainfall, Rainy_Days_Count, Rainy_Days_Percentage, Dry_Days_Count
- Wind: Avg_Wind_Speed, Avg_Wind_Gusts, Wind_Intensity_Index, High_Wind_Days
- Cloud: Avg_Cloud_Cover, Clear_Days, Cloudy_Days, Partly_Cloudy_Days
- Extremes: Extreme_Heat_Days, Cold_Days, Heavy_Rain_Days, Weather_Comfort_Index
- Market: Hottest_Market, Coldest_Market, Wettest_Market

**Correlation (7 measures)**
- Price_Temp_Correlation_Score
- Weather_Impact_on_Price
- Rain_Impact_on_Price
- Weather_Severity_Rating
- Price_KPI
- Dynamic_Title

All measures stored in **_Measures** table for organization.

---